In [4]:
import dash
import dash_mantine_components as dmc
from dash import html, dcc, Input, Output, State
import plotly.graph_objects as go
import plotly.express as px

import webbrowser
import pandas as pd

In [5]:
df = pd.read_csv("combined_processed.csv")

In [2]:
# Having two graphs with a dropdown test

app = dash.Dash()   
df = pd.read_csv("combined_processed.csv")

df["absolute_bidding_amount"] = df["bidding_amount"].abs()

def create_graph():
    fig = go.Figure(px.histogram(df, x = "template"))
    fig.update_layout(title = 'Map usage', xaxis_title = 'Map', yaxis_title = 'Amount', #modebar_remove=['zoom', 'pan'],
                     xaxis={'fixedrange':True}, yaxis={'fixedrange':True})
    return fig  

def create_graph2():
    fig = go.Figure(px.histogram(df, x = "absolute_bidding_amount"))
    fig.update_layout(title = 'Bidding', xaxis_title = 'Bid', yaxis_title = 'Presence')
    return fig 


@app.callback(
    Output('dd-output-container', 'children'),
    Input('demo-dropdown', 'value')
)
def update_output(value):
    return f'You have selected {value}'
 
app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'Title', 
            style = {'textAlign':'center','marginTop':40,'marginBottom':40}),
    dcc.Graph(id = 'plot1', figure = create_graph()),
    dcc.Graph(id = 'plot1', figure = create_graph2()),
    dcc.Dropdown(['NYC', 'MTL', 'SF'], 'NYC', id='demo-dropdown'),
    html.Div(id='dd-output-container')
    ])

In [3]:
# Setup for a search function in the current graph
app = dash.Dash(__name__)   # Initialising dash app
df = pd.read_csv("combined_processed.csv")

app.layout = html.Div([dcc.Graph(id="interactive graph"),
                       dcc.Dropdown(list(df.columns), value = "player_one", id="interactive dropdown"),
                       dcc.Input(id="searcher input", value = "SDK_Ravciozo", type="text"),
                       html.Div(id="searcher output", children="text")])

@app.callback(
    Output("interactive graph", "figure"),
    Input("interactive dropdown", "value"))
def update_figure(column):
    print(column)
    fig = go.Figure(px.histogram(df, x = column))
    fig.update_layout(title = f'{column} Histogram', xaxis_title = column, yaxis_title = 'Amount')

    return fig

@app.callback(
    Output("searcher output", "children"),
    Input("searcher input", "value"),
    Input("interactive dropdown", "value"))
def update_figure(search, column):
    res = str(sum(df[column] == search))
    return f"The amount of {column} for {search} is {res}"

In [4]:
# Interactive Scatterplot

app = dash.Dash(__name__)   # Initialising dash app
df = pd.read_csv("combined_processed.csv")

df["absolute_bidding_amount"] = df["bidding_amount"].abs()

app.layout = html.Div([dcc.Graph(id="interactive graph"),
                       dcc.Dropdown(list(df.columns), value = "score_player_one", id="first dropdown"),
                       dcc.Dropdown(list(df.columns), value = "score_player_one", id="second dropdown")])

@app.callback(
    Output("interactive graph", "figure"),
    Input("first dropdown", "value"),
    Input("second dropdown", "value"))
def update_figure(column1, column2):
    fig = go.Figure(px.scatter(df, x = column1, y = column2, opacity=0.5))
    fig.update_layout(title = f'{column1} vs {column2}', xaxis_title = column1, yaxis_title = column2)

    return fig

In [5]:
# Interactive Heatmap

app = dash.Dash(__name__)   # Initialising dash app
df = pd.read_csv("combined_processed.csv")

df["absolute_bidding_amount"] = df["bidding_amount"].abs()

app.layout = html.Div([dcc.Graph(id="interactive graph"),
                       dcc.Dropdown(list(df.columns), value = "score_player_one", id="first dropdown"),
                       dcc.Dropdown(list(df.columns), value = "score_player_one", id="second dropdown")])

@app.callback(
    Output("interactive graph", "figure"),
    Input("first dropdown", "value"),
    Input("second dropdown", "value"))
def update_figure(column1, column2):
    fig = go.Figure(px.density_heatmap(df, x = column1, y = column2))
    fig.update_layout(title = f'{column1} vs {column2}', xaxis_title = column1, yaxis_title = column2)

    return fig

In [6]:
# Interactive Scatterplot ++ (with filtering)

app = dash.Dash(__name__)   # Initialising dash app
df = pd.read_csv("combined_processed.csv")

df["absolute_bidding_amount"] = df["bidding_amount"].abs()

app.layout = html.Div([dcc.Graph(id="interactive graph"),
                       dcc.Dropdown(list(df.columns), value = "score_player_one", id="first dropdown"),
                       dcc.Dropdown(list(df.columns), value = "score_player_one", id="second dropdown"),
                       dmc.Checkbox(label = "filter", checked = False, radius = "xl", id="filter checkbox"),
                       html.Div([dcc.Dropdown(list(df.columns), value = "score_player_one", id="filter dropdown"),
                                 dcc.Dropdown(sorted(list(set(df["score_player_one"]))), 
                                              value = sorted(list(set(df["score_player_one"])))[0], id="filter dropdown2")],
                                style = {"display": "block"})])

@app.callback(
    Output("interactive graph", "figure"),
    Input("first dropdown", "value"),
    Input("second dropdown", "value"),
    Input("filter checkbox", "checked"),
    Input("filter dropdown", "value"),
    Input("filter dropdown2", "value"))
def update_figure(column1, column2, to_filter, filter1, filter2):
    if to_filter: new_df = df[df[filter1] == filter2]
    else: new_df = df
    fig = go.Figure(px.scatter(new_df, x = column1, y = column2, opacity=0.5))
    fig.update_layout(title = f'{column1} vs {column2}', xaxis_title = column1, yaxis_title = column2)

    return fig

@app.callback(
    Output("filter dropdown", "style"),
    Output("filter dropdown2", "style"),
    Input("filter checkbox", "checked"))
def show_filters(checkbox):
    if checkbox: result = "block"
    else: result = "none"
    return {"display": result}, {"display": result}

@app.callback(
    Output("filter dropdown2", "options"),
    Output("filter dropdown2", "value"),
    Input("filter dropdown", "value"))
def update_second_filter(column):
    selection = sorted(list(set(df[column])))
    return selection, selection[0]

In [7]:
webbrowser.open('http://127.0.0.1:8050/')
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Mar/2023 12:20:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2023 12:20:20] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2023 12:20:20] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2023 12:20:20] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2023 12:20:20] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2023 12:20:20] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2023 12:20:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2023 12:20:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2023 12:20:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2023 12:20:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2023 12:20:22] "POST /_dash-update-component HTTP/1.